# ProbGAN - Synthetic FX + Equity Forecasting Notebook
This notebook uses the `FXEQGenerator` from the ProbGAN repo to generate synthetic price paths based on FX and equity data.

In [1]:
# 📦 Environment Setup
import torch
import numpy as np
import pandas as pd
from train_utils import plot_result
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ModuleNotFoundError: No module named 'models'

In [ ]:
# 📈 Load and Preprocess Data
df = pd.read_csv("raw (FX + EQ).csv")
print(df.shape)
df.head()

In [ ]:
# 🧠 Load Pre-trained Generator
G = FXEQGenerator(z_size=100).to(device)
G.eval()

In [ ]:
# 🔮 Generate Synthetic Forecast
LAST_KNOWN_PRICES = np.array([
    15.136, 20.3352, 17.563, 8.6973, 1.1579, 1.3435,
    20.716, 141.5, 164.252, 9.95, 30.14, 2004.0
])

def generate_synthetic_forecast(G, forecast_horizon=300, n_samples=1, steps_per_pass=20):
    n_passes = forecast_horizon // steps_per_pass
    all_returns = []

    with torch.no_grad():
        for _ in range(n_passes):
            z = torch.randn(n_samples, 100).to(device)
            returns_block = G(z).cpu().numpy()
            all_returns.append(returns_block)

    returns = np.concatenate(all_returns, axis=1)

    prices = []
    for i in range(returns.shape[2]):
        series = returns[0, :, i]
        series_prices = [LAST_KNOWN_PRICES[i]]
        for r in series:
            series_prices.append(series_prices[-1] * (1 + r))
        prices.append(series_prices[1:])

    df_synthetic = pd.DataFrame(np.array(prices).T, columns=df.columns)
    df_synthetic.to_csv("synthetic_prices.csv", index=False)
    return df_synthetic

synthetic_df = generate_synthetic_forecast(G)
synthetic_df.head()

In [ ]:
# 📊 Plot One Sample
fixed_noise = torch.randn(1, 100).to(device)
plot_result(G, fixed_noise, image_size=1, num_epoch=0, save_dir=".", n_series=12)